<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#读取数据" data-toc-modified-id="读取数据-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>读取数据</a></span></li><li><span><a href="#反查表" data-toc-modified-id="反查表-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>反查表</a></span></li><li><span><a href="#协同过滤" data-toc-modified-id="协同过滤-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>协同过滤</a></span><ul class="toc-item"><li><span><a href="#链接本页的页也链接了" data-toc-modified-id="链接本页的页也链接了-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>链接本页的页也链接了</a></span></li><li><span><a href="#这些页也链接了本页的链接的页" data-toc-modified-id="这些页也链接了本页的链接的页-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>这些页也链接了本页的链接的页</a></span></li><li><span><a href="#清除内存" data-toc-modified-id="清除内存-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>清除内存</a></span></li></ul></li><li><span><a href="#保存所有结果" data-toc-modified-id="保存所有结果-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>保存所有结果</a></span></li><li><span><a href="#读取" data-toc-modified-id="读取-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>读取</a></span><ul class="toc-item"><li><span><a href="#链接到本页的页也链接了" data-toc-modified-id="链接到本页的页也链接了-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>链接到本页的页也链接了</a></span></li><li><span><a href="#这些页也链接了本页的链接的页" data-toc-modified-id="这些页也链接了本页的链接的页-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>这些页也链接了本页的链接的页</a></span></li></ul></li><li><span><a href="#Benchmarks" data-toc-modified-id="Benchmarks-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Benchmarks</a></span><ul class="toc-item"><li><span><a href="#hash_unique.unique" data-toc-modified-id="hash_unique.unique-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>hash_unique.unique</a></span></li><li><span><a href="#numpy.unique" data-toc-modified-id="numpy.unique-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>numpy.unique</a></span></li></ul></li></ul></div>

# 读取数据

读取已经解析好的维基百科数据。参见PageRank笔记。

In [1]:
import wiki
wiki_titles = wiki.WikiTitles(file = "pass1.5.data")
wiki_links = wiki.WikiLinks(file = "pass2.npz", wiki_titles=wiki_titles)
print(wiki_links.get_links_from_title("PageRank"))

['Logarithmic scale', 'Algorithm', 'Google Search', 'Web page', 'Web search engine', 'Larry Page', 'Google Patents', 'Network theory', 'Weighting', 'Hyperlink', 'Set (abstract data type)', 'World Wide Web', 'Link building', 'Webgraph', 'CNN', 'USA.gov', 'Recursion', 'Backlink', 'HITS algorithm', 'Jon Kleinberg', 'Teoma', 'Ask.com', 'CLEVER project', 'TrustRank', 'Google Hummingbird', 'Eigenvalues and eigenvectors', 'Scientometrics', 'Thomas L. Saaty', 'Analytic hierarchy process', 'Cognitive model', 'Baidu', 'Robin Li', 'The New York Times', 'Forbes', 'Sergey Brin', 'Stanford University', 'Rajeev Motwani', 'Terry Winograd', 'Google', 'Software patent', 'Citation analysis', 'Eugene Garfield', 'Hyper Search', 'Massimo Marchiori', 'University of Padua', 'Probability distribution', 'Matt Cutts', 'Markov chain', 'URL', 'Adjacency matrix', 'Stochastic matrix', 'Eigenvector centrality', 'Eigengap', 'Expected value', 'Wikipedia', 'Link farm', 'Trade secret', 'Power iteration', 'Steady state', 

In [2]:
import pickle
with open("page_rank.data", "rb") as f:
    PR = pickle.load(f)
    
index = wiki_titles.get_index_from_title("USA")
print(PR[index])

10132.655316718961


# 反查表

In [3]:
import CF
inverse_links = CF.InverseLinks(links = wiki_links, titles = wiki_titles)
print(inverse_links.get_links_from_title_inverse("PageRank"))

['Carl Linnaeus', 'Google Search', 'History of the Internet', 'List of algorithms', 'Web crawler', 'Spamdexing', 'Hyperlink', 'South East England', 'Advogato', 'Markov chain', 'Larry Page', 'Sergey Brin', "Bradford's law", 'Search engine optimization', 'Google Answers', 'Spectrum of a matrix', 'Link farm', 'Google bombing', 'PR', 'MediaWiki', 'Google (verb)', '1996 in science', 'Newsgroup spam', 'Wassily Leontief', 'Timeline of algorithms', 'Relevance', 'Fixed point (mathematics)', 'Global Multimedia Protocols Group', 'Thomas L. Saaty', 'Spam in blogs', 'Backlink', 'MozDex', 'Google Toolbar', 'Automatic summarization', 'Webometrics', 'Affiliate marketing', 'Document retrieval', 'Network theory', 'Web mining', 'List of eponyms (L–Z)', 'Trust metric', 'List of University of California, Berkeley alumni', 'VoIP spam', 'Findability', 'Google', 'Bibliometrics', 'Citation analysis', 'Jon Kleinberg', 'Eugene Garfield', 'Full-text search', 'Referrer spam', 'Centrality', 'Ranking', 'List of Goog

# 协同过滤

In [34]:
import CF
itemCF = CF.ItemCF(links = wiki_links, inverse_links = inverse_links, titles = wiki_titles, weights = np.power(PR, 0.01))

## 链接本页的页也链接了

In [35]:
def print___pages_link_to_this_page_also_link_to(this_page):
    i = wiki_titles.get_index_from_title(this_page)
    #uns, ws = itemCF.pages_link_to_this_page_also_link_to(i, use_weights_inter=True, use_weights_final=True)
    uns, ws = itemCF.pages_link_to_this_page_also_link_to(i, use_weights_inter=True)
    #uns, ws = itemCF.pages_link_to_this_page_also_link_to(i, use_weights_final=True)
    titles = [wiki_titles.get_title_from_index(pid) for pid in uns]
    for page, w in zip(titles, ws):
        print("%-50s %.2f"%(page, w))
        
print___pages_link_to_this_page_also_link_to(this_page="Beijing")
print("====")
print___pages_link_to_this_page_also_link_to(this_page="United States")
print("====")
print___pages_link_to_this_page_also_link_to(this_page="PageRank")
print("====")
print___pages_link_to_this_page_also_link_to(this_page="Liang (surname)")

2008 Summer Olympics                               0.28
China                                              0.24
2015 World Championships in Athletics              0.23
NBC Olympic broadcasts                             0.21
Athletics at the 2008 Summer Olympics              0.21
Shanghai                                           0.20
Tianjin                                            0.17
Guangzhou                                          0.15
Athletics at the 2016 Summer Olympics              0.14
Beijing Subway                                     0.14
====
New York City                                      0.16
California                                         0.16
Canada                                             0.14
United Kingdom                                     0.13
New York (state)                                   0.13
Washington, D.C.                                   0.12
Texas                                              0.12
Los Angeles                                

## 这些页也链接了本页的链接的页

In [36]:
def print__pages_this_page_link_to_also_linked_by(this_page):
    i = wiki_titles.get_index_from_title(this_page)
    uns, ws = itemCF.pages_this_page_link_to_also_linked_by(i, use_weights_inter=True)
    titles = [wiki_titles.get_title_from_index(pid) for pid in uns]
    for page, w in zip(titles, ws):
        print("%-50s %.2f"%(page, w))


print__pages_this_page_link_to_also_linked_by(this_page="Beijing")
print("====")
print__pages_this_page_link_to_also_linked_by(this_page="United States")
print("====")
print__pages_this_page_link_to_also_linked_by(this_page="PageRank")
print("====")
print__pages_this_page_link_to_also_linked_by(this_page="Liang (surname)")

History of Beijing                                 0.31
Geography of Beijing                               0.26
List of Beijing landmarks                          0.26
Hebei                                              0.22
Transport in Beijing                               0.21
Dongcheng District, Beijing                        0.21
China                                              0.20
Beijing city fortifications                        0.19
List of Major National Historical and Cultural Sites in Beijing 0.19
Beijing Subway                                     0.18
====
History of the United States                       0.23
Index of United States-related articles            0.22
Culture of the United States                       0.21
Americans                                          0.19
Economy of the United States                       0.18
Demographics of the United States                  0.18
Outline of the United States                       0.16
Political divisions of the Uni

## 清除内存

In [8]:
del itemCF
del wiki_links
del wiki_titles
del inverse_links

# 保存所有结果

对所有article，计算协同过滤top10，并且储存。

我们采用多进程方案，并且多进程采用`spawn`方式。进程采用`spawn`方式，尽管可能需要更多的时间进行任务的初始化，但是兼容windows。

In [ ]:
%%file wiki_save.py
import multiprocessing
import wiki
import CF
import numpy as np
import time

def map1(tid, start, end):
    print("thread %02d: [%d - %d]"%(tid, start, end))
    # initialization need lots of memory, we avoid running them at the same time
    if tid < 8:
        for i in range(tid):
            print("thread %02d: sleeping"%tid)        
            time.sleep(20)        
    print("thread %02d: initializing"%tid)
    itemCF = CF.ItemCF(wiki_titles=None)
    n_pages = end - start
    pages_link_to_this_page_also_link_to_ = np.zeros((n_pages,10), np.int)-1
    for i in range(n_pages):
        ids, ws = itemCF.pages_link_to_this_page_also_link_to(start + i)
        pages_link_to_this_page_also_link_to_[i, 0:len(ids)] = ids
        if i % 1000 == 0:
            print("thread %02d: %d"%(tid, i))
        
    return pages_link_to_this_page_also_link_to_

def map2(tid, start, end):
    print("thread %02d: [%d - %d]"%(tid, start, end))
    # initialization need lots of memory, we avoid running them at the same time
    if tid < 8:
        for i in range(tid):
            print("thread %02d: sleeping"%tid)        
            time.sleep(20)        
    print("thread %02d: initializing"%tid)
    
    itemCF = wiki_item_CF.ItemCF(wiki_titles=None)
    n_pages = end - start
    pages_this_page_link_to_also_linked_by_ = np.zeros((n_pages,10), np.int)-1
    for i in range(n_pages):
        ids, ws = itemCF.pages_this_page_link_to_also_linked_by(start + i)
        pages_this_page_link_to_also_linked_by_[i, 0:len(ids)] = ids
        if i % 1000 == 0:
            print("thread %02d: %d"%(tid, i))
        
    return pages_this_page_link_to_also_linked_by_
    

def save(n_pages):
        
    n_proc = 8
    n_splits = 32
    pool = multiprocessing.Pool(processes = n_proc)

    results = pool.starmap(map1, [(i, i*n_pages//n_splits, (i+1)*n_pages//n_splits) for i in range(n_splits)])    
    pages_link_to_this_page_also_link_to_ = np.concatenate(results)
    np.savez_compressed("pages_link_to_this_page_also_link_to.npz", pages_link_to_this_page_also_link_to_)
    
    results = pool.starmap(map2, [(i, i*n_pages//n_splits, (i+1)*n_pages//n_splits) for i in  range(n_splits)])
    pages_this_page_link_to_also_linked_by_ = np.concatenate(results)
    np.savez_compressed("pages_this_page_link_to_also_linked_by.npz", pages_this_page_link_to_also_linked_by_)



In [ ]:
import wiki
import wiki_save
import imp
imp.reload(wiki_save)
imp.reload(wiki)
n_pages = len(wiki.WikiLinks().page_n_links)
%timeit -r1 -n1 wiki_save.save(n_pages)

# 读取

In [1]:
import CF
import wiki
wiki_titles = wiki.WikiTitles()
itemCFCached=CF.ItemCFCached(pages_link_to_this_page_also_link_to="pages_link_to_this_page_also_link_to.npz",
                            pages_this_page_link_to_also_linked_by="pages_this_page_link_to_also_linked_by.npz",
                            titles = wiki_titles)

## 链接到本页的页也链接了

In [5]:
def print___pages_link_to_this_page_also_link_to(this_page):
    titles = itemCFCached.pages_link_to_this_page_also_link_to_title(this_page)
    for page in titles:
        print("%-50s"%(page))
        
print___pages_link_to_this_page_also_link_to(this_page="Beijing")
print("====")
print___pages_link_to_this_page_also_link_to(this_page="PageRank")
print("====")
print___pages_link_to_this_page_also_link_to(this_page="Liang (surname)")

%timeit itemCFCached.pages_link_to_this_page_also_link_to_title("Beijing")

2008 Summer Olympics                              
China                                             
2015 World Championships in Athletics             
Athletics at the 2008 Summer Olympics             
NBC Olympic broadcasts                            
Shanghai                                          
Tianjin                                           
Guangzhou                                         
Athletics at the 2016 Summer Olympics             
Beijing Subway                                    
====
HITS algorithm                                    
Spamdexing                                        
Backlink                                          
Google Panda                                      
Google Penguin                                    
TrustRank                                         
Katz centrality                                   
History of Google                                 
Search engine optimization                        
Larry Page                

## 这些页也链接了本页的链接的页

In [3]:
def print__pages_this_page_link_to_also_linked_by(this_page):
    titles = itemCFCached.pages_this_page_link_to_also_linked_by_title(this_page)
    for page in titles:
        print("%-50s"%(page))

print__pages_this_page_link_to_also_linked_by(this_page="Beijing")
print("====")
print__pages_this_page_link_to_also_linked_by(this_page="United States")
print("====")
print__pages_this_page_link_to_also_linked_by(this_page="PageRank")
print("====")
print__pages_this_page_link_to_also_linked_by(this_page="Liang (surname)")

History of Beijing                                
Geography of Beijing                              
List of Beijing landmarks                         
Transport in Beijing                              
Hebei                                             
Dongcheng District, Beijing                       
List of Major National Historical and Cultural Sites in Beijing
Beijing city fortifications                       
China                                             
Beijing Subway                                    
====
History of the United States                      
Index of United States-related articles           
Culture of the United States                      
Americans                                         
Economy of the United States                      
Demographics of the United States                 
Outline of the United States                      
Political divisions of the United States          
Outline of United States history                  
Race and ethn

# Benchmarks

## hash_unique.unique

In [86]:
def print___pages_link_to_this_page_also_link_to(this_page):
    i = wiki_titles.get_index_from_title(this_page)
    uns, ws = itemCF.pages_link_to_this_page_also_link_to(i)
    titles = [wiki_titles.get_title_from_index(pid) for pid in uns]
    for page, w in zip(titles, ws):
        print("%-50s %.2f"%(page, w))
        
%timeit itemCF.pages_link_to_this_page_also_link_to(wiki_titles.get_index_from_title("Beijing"))
%timeit itemCF.pages_link_to_this_page_also_link_to(wiki_titles.get_index_from_title("USA"))
import cProfile
cProfile.run('itemCF.pages_link_to_this_page_also_link_to(wiki_titles.get_index_from_title("Beijing"))')

63.6 ms ± 784 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
553 ms ± 9.93 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
         26 function calls in 0.069 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.004    0.004 <__array_function__ internals>:2(argpartition)
        1    0.000    0.000    0.000    0.000 <__array_function__ internals>:2(argsort)
        1    0.000    0.000    0.069    0.069 <string>:1(<module>)
        2    0.000    0.000    0.004    0.002 fromnumeric.py:52(_wrapfunc)
        1    0.000    0.000    0.000    0.000 fromnumeric.py:752(_argpartition_dispatcher)
        1    0.000    0.000    0.004    0.004 fromnumeric.py:756(argpartition)
        1    0.000    0.000    0.000    0.000 fromnumeric.py:995(_argsort_dispatcher)
        1    0.000    0.000    0.000    0.000 fromnumeric.py:999(argsort)
        1    0.000    0.000    0.045    0.045 hash_unique.py:3

## numpy.unique

In [92]:
def print___pages_link_to_this_page_also_link_to(this_page):
    i = wiki_titles.get_index_from_title(this_page)
    uns, ws = itemCF.pages_link_to_this_page_also_link_to(i)
    titles = [wiki_titles.get_title_from_index(pid) for pid in uns]
    for page, w in zip(titles, ws):
        print("%-50s %.2f"%(page, w))
        
%timeit itemCF.pages_link_to_this_page_also_link_to(wiki_titles.get_index_from_title("Beijing"))
%timeit itemCF.pages_link_to_this_page_also_link_to(wiki_titles.get_index_from_title("USA"))
import cProfile
cProfile.run('itemCF.pages_link_to_this_page_also_link_to(wiki_titles.get_index_from_title("Beijing"))')

124 ms ± 368 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
1.02 s ± 5.36 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
         57 function calls (54 primitive calls) in 0.125 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.003    0.003 <__array_function__ internals>:2(argpartition)
        1    0.000    0.000    0.000    0.000 <__array_function__ internals>:2(argsort)
        1    0.000    0.000    0.001    0.001 <__array_function__ internals>:2(concatenate)
        1    0.000    0.000    0.001    0.001 <__array_function__ internals>:2(diff)
        1    0.000    0.000    0.004    0.004 <__array_function__ internals>:2(nonzero)
        1    0.000    0.000    0.109    0.109 <__array_function__ internals>:2(unique)
        1    0.000    0.000    0.125    0.125 <string>:1(<module>)
        3    0.000    0.000    0.000    0.000 _asarray.py:86(asanyarray)
        1    0.000    